In [1]:
import pandas as pd
import pymaid

rm = pymaid.CatmaidInstance('server_url', 'http_user', 'http_pw', 'token')

In [2]:
n = pymaid.get_neuron(16)

In [3]:
# Get all connectors
cn = n.connectors

# Add a column with the URL to coordinates
cn['URL'] = pymaid.url_to_coordinates(cn, 
                                      stack_id = 5,
                                      active_node_id = cn.connector_id.values)
cn.head()

,treenode_id,connector_id,relation,x,y,z,URL
0,127234,999188,0,438817,164242,217440,https://neuropil.janelia.org/tracing/fafb/v14/...
1,2591,97954,0,437120,160998,211920,https://neuropil.janelia.org/tracing/fafb/v14/...
2,2665,98300,0,437184,162324,214880,https://neuropil.janelia.org/tracing/fafb/v14/...
3,2646,98373,0,437042,162452,214120,https://neuropil.janelia.org/tracing/fafb/v14/...
4,2654,98415,0,436761,163690,214440,https://neuropil.janelia.org/tracing/fafb/v14/...


In [4]:
cn.to_csv('filename.csv')

In [5]:
cn.to_clipboard()

In [6]:
pre = cn[cn.relation == 0]

In [7]:
ran = pre.sample(frac=1)
ran.head()

,treenode_id,connector_id,relation,x,y,z,URL
868,10350265,10350269,0,358255,151135,167240,https://neuropil.janelia.org/tracing/fafb/v14/...
1423,10366582,10366581,0,336551,159297,177360,https://neuropil.janelia.org/tracing/fafb/v14/...
785,10345534,10345533,0,375162,146617,170760,https://neuropil.janelia.org/tracing/fafb/v14/...
597,10337834,10337835,0,385421,134986,173200,https://neuropil.janelia.org/tracing/fafb/v14/...
1591,10370697,10370695,0,341188,153709,169440,https://neuropil.janelia.org/tracing/fafb/v14/...


In [8]:
post_details = pymaid.get_connector_details(n.postsynapses)
post_details.head()

INFO  : Data for 412 of 412 unique connector IDs retrieved (pymaid)


,connector_id,presynaptic_to,postsynaptic_to,presynaptic_to_node,postsynaptic_to_node
0,474311,203840,"[7911576, 8638, 16, 6483677]",474312,"[2736161, 474295, 124991, 21109788]"
1,573905,54753,"[7634, 279469, 30358, 17238, 16, 597080, 39182...",573904,"[573902, 1482460, 1670909, 2875768, 127266, 28..."
2,1335332,55153,"[8408275, 16]",1326548,"[27245140, 125650]"
3,1482374,203840,"[10379190, 9052688, 279469, 597085, 597100, 59...",1277055,"[2875773, 30223253, 1482362, 2875771, 2875774,..."
4,1510194,288558,"[16, 203840]",1510193,"[124597, 1510195]"


In [9]:
post_details[post_details.presynaptic_to_node.isnull()]

,connector_id,presynaptic_to,postsynaptic_to,presynaptic_to_node,postsynaptic_to_node
401,20041670,None,"[4954519, 16]",None,"[20041667, 18719343]"


In [10]:
# Now get the locations
locs = pymaid.get_node_location(post_details.presynaptic_to_node.values)
locs.head()

,node_id,x,y,z
0,10362880,367320.0,152292.0,177760.0
1,18804742,466093.0,234873.0,38560.0
2,18802702,466727.0,233599.0,33640.0
3,10342419,381419.0,140309.0,180280.0
4,10315798,345150.0,159327.0,186640.0


In [11]:
cn_merged = pd.merge(post_details.set_index('presynaptic_to_node'),
                     locs.set_index('node_id'),
                     left_index=True, right_index=True)
cn_merged.head()

,connector_id,presynaptic_to,postsynaptic_to,postsynaptic_to_node,x,y,z
437124,7265239,202541,"[8680461, 16, 13782]","[28458449, 1786, 22231380]",438062.0,140370.0,203160.0
437172,7265176,202541,"[30571, 40637, 16, 4205169]","[6428950, 447292, 1819, 7265200]",437010.0,141171.0,204400.0
474312,474311,203840,"[7911576, 8638, 16, 6483677]","[2736161, 474295, 124991, 21109788]",421574.0,127771.0,194720.0
573904,573905,54753,"[7634, 279469, 30358, 17238, 16, 597080, 39182...","[573902, 1482460, 1670909, 2875768, 127266, 28...",434460.0,165646.0,216160.0
1275576,2873212,203840,"[595239, 16]","[2873213, 127687]",436480.0,147325.0,221360.0


In [12]:
cn_merged['url'] = pymaid.url_to_coordinates(cn_merged, stack_id=5)
cn_merged['neuron_name'] = cn_merged.presynaptic_to.astype(str).map(pymaid.get_names(cn_merged.presynaptic_to.values))
cn_merged['has_soma'] = cn_merged.presynaptic_to.map(pymaid.has_soma(cn_merged.presynaptic_to.values))

cn_merged.head()

,connector_id,presynaptic_to,postsynaptic_to,postsynaptic_to_node,x,y,z,url,neuron_name,has_soma
437124,7265239,202541,"[8680461, 16, 13782]","[28458449, 1786, 22231380]",438062.0,140370.0,203160.0,https://neuropil.janelia.org/tracing/fafb/v14/...,putative Gad1-F-600202 40613 AJ,True
437172,7265176,202541,"[30571, 40637, 16, 4205169]","[6428950, 447292, 1819, 7265200]",437010.0,141171.0,204400.0,https://neuropil.janelia.org/tracing/fafb/v14/...,putative Gad1-F-600202 40613 AJ,True
474312,474311,203840,"[7911576, 8638, 16, 6483677]","[2736161, 474295, 124991, 21109788]",421574.0,127771.0,194720.0,https://neuropil.janelia.org/tracing/fafb/v14/...,APL 203841 MR JSL,True
573904,573905,54753,"[7634, 279469, 30358, 17238, 16, 597080, 39182...","[573902, 1482460, 1670909, 2875768, 127266, 28...",434460.0,165646.0,216160.0,https://neuropil.janelia.org/tracing/fafb/v14/...,neuron 54754,False
1275576,2873212,203840,"[595239, 16]","[2873213, 127687]",436480.0,147325.0,221360.0,https://neuropil.janelia.org/tracing/fafb/v14/...,APL 203841 MR JSL,True


In [13]:
cn_merged = cn_merged[['presynaptic_to', 'neuron_name', 'has_soma', 'connector_id', 'x', 'y', 'z', 'url']]
cn_merged.head()

,presynaptic_to,neuron_name,has_soma,connector_id,x,y,z,url
437124,202541,putative Gad1-F-600202 40613 AJ,True,7265239,438062.0,140370.0,203160.0,https://neuropil.janelia.org/tracing/fafb/v14/...
437172,202541,putative Gad1-F-600202 40613 AJ,True,7265176,437010.0,141171.0,204400.0,https://neuropil.janelia.org/tracing/fafb/v14/...
474312,203840,APL 203841 MR JSL,True,474311,421574.0,127771.0,194720.0,https://neuropil.janelia.org/tracing/fafb/v14/...
573904,54753,neuron 54754,False,573905,434460.0,165646.0,216160.0,https://neuropil.janelia.org/tracing/fafb/v14/...
1275576,203840,APL 203841 MR JSL,True,2873212,436480.0,147325.0,221360.0,https://neuropil.janelia.org/tracing/fafb/v14/...


In [14]:
cn_merged.sample(frac=1).to_clipboard()